In [1]:
from browsermobproxy import Server
import time
import urllib.request, json
from pandas import *
from selenium import webdriver
server = Server("/Users/joono/browsermob-proxy-2.1.4/bin/browsermob-proxy")
server.start()
proxy = server.create_proxy()


In [2]:
keyword = '혜화역'
webdriver_path = '/Users/joono/chromedriver'
email = 'jjjjooonno@gmail.com'

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server={host}:{port}'.format(host='localhost', port=proxy.port))

In [4]:
dr = webdriver.Chrome(webdriver_path,chrome_options=chrome_options)
proxy.new_har("zigbang", options={'captureHeaders': True, 'captureContent': True})

(204, None)

In [5]:
dr.get("https://www.zigbang.com/")
time.sleep(1)
dr.find_element_by_xpath('/html/body/div[1]/div[1]/div[2]/div/button[1]').click()
time.sleep(3)
dr.find_element_by_name('username').send_keys(email)
time.sleep(1)
dr.find_element_by_xpath('/html/body/div[4]/div/form/div[2]/div[2]/button').click()
time.sleep(2)

In [8]:
# dr.find_element_by_id('room-textfield').send_keys(keyword)
# time.sleep(5)
# dr.find_element_by_xpath('//*[@id="search_btn"]').click()
# time.sleep(10)
url_json_holder = []
for ent in proxy.har['log']['entries']:
    url_json = str(ent['request']['url'])
    if 'https://api.zigbang.com/v3/items?detail=true&item_ids=' in url_json:
        url_json_holder.append(url_json)
        break

In [9]:
rent = []
deposit = []
floor = []
floor_all = []
size = []
near_subways = []
elevator = []
movein_date = []
description = []
user_has_penalty = []
with urllib.request.urlopen(url_json_holder[0]) as url:
    data = json.loads(url.read().decode())
for i in data['items']:
    rent.append(i['item']['rent'])
    deposit.append(i['item']['deposit'])
    floor.append(i['item']['floor'])
    floor_all.append(i['item']['floor_all'])
    size.append(i['item']['size'])
    near_subways.append(i['item']['near_subways'])
    elevator.append(i['item']['elevator'])
    movein_date.append(i['item']['movein_date'])
    description.append(i['item']['description'])
    user_has_penalty.append(i['item']['user_has_penalty'])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: pandas.json is deprecated and will be removed in a future version.
You can access loads as pandas.io.json.loads
  if sys.path[0] == '':


In [10]:
zigbangs = DataFrame({'월세':rent,'보증금/전세':deposit,'층':floor,'건물층수':floor_all,'평수':size,'가까운 역':near_subways,
                      '입주날짜':movein_date,'설명':description,'판매자벌점여부':user_has_penalty})
zigbangs.to_csv('zigbang_{0}.csv'.format(keyword+'test'))